In [131]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [132]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [133]:
tf.keras.backend.clear_session()

In [134]:
subject = 'Maranhão - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [135]:
data = pd.read_csv('2003_mo_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - Consumo de Cimento (t)
0,2003-1,23.368400,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,8.213662,2.330394,1005.666766,5776.461493,26.995403,22.910306,79.260234,2.399209,0.260097,0.635570,4.315322e+07,2.636001e+06,6.034057,3.872495e+07,43.342
1,2003-2,23.167977,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,8.207986,2.330957,1005.664945,5776.461493,26.995348,22.913026,79.273296,2.397614,0.258964,0.635997,4.320832e+07,2.636851e+06,6.036083,3.874405e+07,28.663
2,2003-3,23.563273,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,8.202310,2.331285,1005.663882,5776.461493,26.995316,22.914612,79.280916,2.396684,0.257352,0.636424,4.326343e+07,2.637701e+06,6.038110,3.876316e+07,27.900
3,2003-4,24.008345,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,8.196633,2.331321,1005.663768,5776.461493,26.995312,22.914782,79.281732,2.396584,0.255929,0.636851,4.331853e+07,2.638551e+06,6.040136,3.878226e+07,25.739
4,2003-5,24.366896,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,8.190957,2.397956,1005.413091,6468.525319,26.849142,22.998258,80.364896,2.323932,0.254109,0.637278,4.337364e+07,2.639401e+06,6.042163,3.880136e+07,30.203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,31.724066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.529647,1005.758478,1486.160102,26.650498,22.818106,80.388377,1.916079,0.520178,NaN,NaN,NaN,NaN,NaN,102.698
236,2022-9,31.679231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.236976,1005.403985,1494.696772,26.858636,22.770108,79.162255,2.051831,0.518515,NaN,NaN,NaN,NaN,NaN,98.133
237,2022-10,31.524870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.475850,1005.164899,1487.321456,26.943065,22.778954,78.817219,2.080726,0.516447,NaN,NaN,NaN,NaN,NaN,94.964
238,2022-11,31.512212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.441295,1005.054471,1467.192232,26.918070,22.810885,79.072190,2.033847,0.513912,NaN,NaN,NaN,NaN,NaN,97.684


In [136]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.196619,0.282587,0.022205,0.105918,-0.712248,-0.605630,0.694542,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.195619,0.280378,0.022205,0.105782,-0.707680,-0.602083,0.691473,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.195036,0.279090,0.022205,0.105703,-0.705016,-0.600014,0.689683,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-0.194974,0.278952,0.022205,0.105695,-0.704731,-0.599792,0.689491,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-0.076690,-0.024928,0.126049,-0.254260,-0.564552,-0.305679,0.549717,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.550687,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,1.189788,-0.316867,0.709498,-0.638954,-0.123881,-1.053413,-0.732207,0.147024,1.221276,1.327369,1.120107,-1.533048,0.771361,0.866519
188,-0.554816,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,1.185037,-0.330364,0.244198,-0.639832,0.044005,-1.019700,-0.793344,0.234950,1.236987,1.321661,1.107960,-1.504452,0.745175,0.846505
189,-0.564631,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,1.180287,-1.098117,0.208362,-0.642297,0.195979,-0.727103,-0.658398,0.187104,1.252847,1.315953,1.095812,-1.475855,0.718990,0.826491
190,-0.596444,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,1.175536,-1.102097,0.167990,-0.647779,-0.053393,-0.316506,-0.225535,-0.031774,1.276241,1.310245,1.083664,-1.447259,0.692804,0.806477


In [137]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Maranhão - Consumo de Cimento (t), Length: 240, dtype: float64

In [138]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.196619,0.282587,0.022205,0.105918,-0.712248,-0.605630,0.694542,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.195619,0.280378,0.022205,0.105782,-0.707680,-0.602083,0.691473,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.195036,0.279090,0.022205,0.105703,-0.705016,-0.600014,0.689683,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-0.194974,0.278952,0.022205,0.105695,-0.704731,-0.599792,0.689491,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-0.076690,-0.024928,0.126049,-0.254260,-0.564552,-0.305679,0.549717,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.796451,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,1.431469,0.495217,0.243276,-0.661819,0.803324,1.080537,0.420152,-0.443237,0.944351,1.437372,1.197811,-1.616416,0.972153,1.061498
158,0.727411,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,1.418921,0.982093,0.704668,-0.664315,0.571787,1.226316,0.656928,-0.746910,0.952621,1.436068,1.200726,-1.633804,0.973222,1.061628
159,0.657765,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,1.406373,0.693908,1.024101,-0.660073,0.415711,1.115871,0.656928,-0.882676,0.960982,1.434764,1.203641,-1.651193,0.974290,1.061759
160,0.591941,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,1.393825,0.812950,1.570300,-0.655535,0.274636,0.737157,0.436286,-0.784425,0.969363,1.433461,1.206556,-1.668581,0.975359,1.061889


In [139]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
157    66.661
158    78.801
159    67.256
160    84.280
161    86.501
Name: Maranhão - Consumo de Cimento (t), Length: 162, dtype: float64

In [140]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [141]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [142]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
126,1.796771,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,0.858960,-0.542428,0.686896,-0.630242,-0.200400,-0.448745,-0.097398,-0.191176,0.673675,0.395980,0.831326,0.271134,1.034804,0.988167
127,1.807797,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,0.892045,-0.938409,0.113657,-0.622498,0.411160,-0.881025,-0.786733,0.338303,0.687412,0.398649,0.850067,0.230720,1.041385,0.998849
128,1.778040,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,0.925130,-0.458288,-0.243074,-0.618407,0.755606,-1.273702,-1.280776,0.711827,0.706891,0.401317,0.868808,0.190305,1.047966,1.009532
129,1.750228,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,0.958215,-0.735620,-0.500551,-0.626137,0.570431,-1.395846,-1.300155,0.704428,0.722784,0.403986,0.887549,0.149891,1.054547,1.020214
130,1.717721,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,0.991300,-0.543447,-0.862060,-0.636342,0.209520,-1.181683,-0.974891,0.415826,0.738712,0.406655,0.906289,0.109477,1.061127,1.030897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.550687,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,1.189788,-0.316867,0.709498,-0.638954,-0.123881,-1.053413,-0.732207,0.147024,1.221276,1.327369,1.120107,-1.533048,0.771361,0.866519
188,-0.554816,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,1.185037,-0.330364,0.244198,-0.639832,0.044005,-1.019700,-0.793344,0.234950,1.236987,1.321661,1.107960,-1.504452,0.745175,0.846505
189,-0.564631,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,1.180287,-1.098117,0.208362,-0.642297,0.195979,-0.727103,-0.658398,0.187104,1.252847,1.315953,1.095812,-1.475855,0.718990,0.826491
190,-0.596444,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,1.175536,-1.102097,0.167990,-0.647779,-0.053393,-0.316506,-0.225535,-0.031774,1.276241,1.310245,1.083664,-1.447259,0.692804,0.806477


In [143]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [144]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [156]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(18,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [157]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [158]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2843332744, 3770364254, 1483413004, 132235550, 1491671449]


Step: 0 ___________________________________________
val_loss: 188.79000854492188
winner_seed: 2843332744


Step: 1 ___________________________________________
val_loss: 169.27313232421875
winner_seed: 3770364254


Step: 2 ___________________________________________
val_loss: 205.0075225830078


Step: 3 ___________________________________________
val_loss: 196.18380737304688


Step: 4 ___________________________________________
val_loss: 155.4315185546875
winner_seed: 1491671449


final_seed: 1491671449


In [159]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 128ms/step - loss: 10846.8604 - val_loss: 6166.9546
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 4373.7607 - val_loss: 1667.5903
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 1355.6346 - val_loss: 790.6194
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 2391.5789 - val_loss: 2296.4597
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 1233.8950 - val_loss: 623.6835
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 636.5801 - val_loss: 518.0097
Epoch 7/10000
4/4 [==============================] - 0s 12ms/step - loss: 535.2404 - val_loss: 465.5885
Epoch 8/10000
4/4 [==============================] - 0s 11ms/step - loss: 471.0919 - val_loss: 377.7406
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 398.6588 - val_loss: 359.2835
Epoch 10/10000
4/4 [==============================] - 

4/4 [==============================] - 0s 12ms/step - loss: 104.0851 - val_loss: 216.5349
Epoch 80/10000
4/4 [==============================] - 0s 12ms/step - loss: 96.8876 - val_loss: 219.3114
Epoch 81/10000
4/4 [==============================] - 0s 12ms/step - loss: 129.0994 - val_loss: 200.1412
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 110.3879 - val_loss: 217.5706
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 104.5599 - val_loss: 240.2158
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 93.6578 - val_loss: 209.1691
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 118.6076 - val_loss: 187.3967
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 109.1378 - val_loss: 202.8312
Epoch 87/10000
4/4 [==============================] - 0s 12ms/step - loss: 81.7341 - val_loss: 190.4669
Epoch 88/10000
4/4 [==============================] - 0s 11ms/step - loss

4/4 [==============================] - 0s 10ms/step - loss: 72.1353 - val_loss: 220.0399
Epoch 158/10000
4/4 [==============================] - 0s 12ms/step - loss: 70.5517 - val_loss: 223.3706
Epoch 159/10000
4/4 [==============================] - 0s 12ms/step - loss: 66.4330 - val_loss: 213.8281
Epoch 160/10000
4/4 [==============================] - 0s 12ms/step - loss: 60.2826 - val_loss: 207.8414
Epoch 161/10000
4/4 [==============================] - 0s 15ms/step - loss: 67.3177 - val_loss: 230.9704
Epoch 162/10000
4/4 [==============================] - 0s 12ms/step - loss: 66.8819 - val_loss: 195.5328
Epoch 163/10000
4/4 [==============================] - 0s 12ms/step - loss: 66.4223 - val_loss: 218.8934
Epoch 164/10000
4/4 [==============================] - 0s 13ms/step - loss: 80.9714 - val_loss: 210.9869
Epoch 165/10000
4/4 [==============================] - 0s 12ms/step - loss: 67.1734 - val_loss: 197.1992
Epoch 166/10000
4/4 [==============================] - 0s 13ms/step - l

4/4 [==============================] - 0s 11ms/step - loss: 52.0430 - val_loss: 226.2398
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 50.5322 - val_loss: 192.0819
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 44.1161 - val_loss: 218.3022
Epoch 238/10000
4/4 [==============================] - 0s 12ms/step - loss: 60.8152 - val_loss: 209.2880
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 61.6253 - val_loss: 222.0045
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 61.8025 - val_loss: 204.5738
Epoch 241/10000
4/4 [==============================] - 0s 11ms/step - loss: 61.7217 - val_loss: 176.4924
Epoch 242/10000
4/4 [==============================] - 0s 11ms/step - loss: 60.3777 - val_loss: 197.5802
Epoch 243/10000
4/4 [==============================] - 0s 12ms/step - loss: 56.1751 - val_loss: 175.4968
Epoch 244/10000
4/4 [==============================] - 0s 12ms/step - lo

4/4 [==============================] - 0s 13ms/step - loss: 47.5907 - val_loss: 218.1626
Epoch 314/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.4026 - val_loss: 193.2729
Epoch 315/10000
4/4 [==============================] - 0s 11ms/step - loss: 41.1138 - val_loss: 193.7417
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step - loss: 36.0991 - val_loss: 207.8944
Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 44.6750 - val_loss: 191.6749
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 33.3747 - val_loss: 185.9089
Epoch 319/10000
4/4 [==============================] - 0s 12ms/step - loss: 35.7345 - val_loss: 182.6167
Epoch 320/10000
4/4 [==============================] - 0s 12ms/step - loss: 36.9083 - val_loss: 204.2686
Epoch 321/10000
4/4 [==============================] - 0s 11ms/step - loss: 41.7798 - val_loss: 197.8343
Epoch 322/10000
4/4 [==============================] - 0s 12ms/step - los

...

4/4 [==============================] - 0s 12ms/step - loss: 21.0368 - val_loss: 235.2993
Epoch 860/10000
4/4 [==============================] - 0s 14ms/step - loss: 18.2905 - val_loss: 232.5996
Epoch 861/10000
4/4 [==============================] - 0s 11ms/step - loss: 15.1846 - val_loss: 228.1324
Epoch 862/10000
4/4 [==============================] - 0s 12ms/step - loss: 15.8511 - val_loss: 229.6357
Epoch 863/10000
4/4 [==============================] - 0s 11ms/step - loss: 18.5755 - val_loss: 231.5533
Epoch 864/10000
4/4 [==============================] - 0s 11ms/step - loss: 20.0952 - val_loss: 244.1427
Epoch 865/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.6501 - val_loss: 230.0601
Epoch 866/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.9188 - val_loss: 247.8398
Epoch 867/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.3827 - val_loss: 244.4268
Epoch 868/10000
4/4 [==============================] - 0s 11ms/step - lo

4/4 [==============================] - 0s 13ms/step - loss: 13.7386 - val_loss: 241.5158
Epoch 938/10000
4/4 [==============================] - 0s 14ms/step - loss: 14.8864 - val_loss: 247.2592
Epoch 939/10000
4/4 [==============================] - 0s 12ms/step - loss: 18.4024 - val_loss: 238.1582
Epoch 940/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.5273 - val_loss: 241.4695
Epoch 941/10000
4/4 [==============================] - 0s 12ms/step - loss: 11.9695 - val_loss: 245.9426
Epoch 942/10000
4/4 [==============================] - 0s 11ms/step - loss: 22.8333 - val_loss: 250.1352
Epoch 943/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.7654 - val_loss: 247.3116
Epoch 944/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.4799 - val_loss: 242.0157
Epoch 945/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.2852 - val_loss: 261.7499
Epoch 946/10000
4/4 [==============================] - 0s 9ms/step - los

4/4 [==============================] - 0s 11ms/step - loss: 23.9892 - val_loss: 258.4760
Epoch 1016/10000
4/4 [==============================] - 0s 11ms/step - loss: 18.3523 - val_loss: 245.1318
Epoch 1017/10000
4/4 [==============================] - 0s 12ms/step - loss: 18.9959 - val_loss: 233.8481
Epoch 1018/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.3083 - val_loss: 224.5458
Epoch 1019/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.8351 - val_loss: 240.6532
Epoch 1020/10000
4/4 [==============================] - 0s 13ms/step - loss: 16.3915 - val_loss: 247.8652
Epoch 1021/10000
4/4 [==============================] - 0s 12ms/step - loss: 15.5963 - val_loss: 243.9709
Epoch 1022/10000
4/4 [==============================] - 0s 12ms/step - loss: 10.8580 - val_loss: 245.8696
Epoch 1023/10000
4/4 [==============================] - 0s 11ms/step - loss: 19.3488 - val_loss: 255.1248
Epoch 1024/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 11ms/step - loss: 20.2628 - val_loss: 283.8184
Epoch 1093/10000
4/4 [==============================] - 0s 12ms/step - loss: 18.8184 - val_loss: 263.9496
Epoch 1094/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.3984 - val_loss: 263.3163
Epoch 1095/10000
4/4 [==============================] - 0s 14ms/step - loss: 11.3501 - val_loss: 261.4672
Epoch 1096/10000
4/4 [==============================] - 0s 13ms/step - loss: 16.1532 - val_loss: 266.6269
Epoch 1097/10000
4/4 [==============================] - 0s 13ms/step - loss: 20.0699 - val_loss: 269.3646
Epoch 1098/10000
4/4 [==============================] - 0s 12ms/step - loss: 18.1670 - val_loss: 299.5776
Epoch 1099/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.0047 - val_loss: 283.5259
Epoch 1100/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.9466 - val_loss: 269.9860
Epoch 1101/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 13ms/step - loss: 20.3536 - val_loss: 266.4565
Epoch 1170/10000
4/4 [==============================] - 0s 12ms/step - loss: 20.2690 - val_loss: 263.2095
Epoch 1171/10000
4/4 [==============================] - 0s 11ms/step - loss: 14.8071 - val_loss: 270.0000
Epoch 1172/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.1651 - val_loss: 270.9359
Epoch 1173/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.0599 - val_loss: 271.9332
Epoch 1174/10000
4/4 [==============================] - 0s 11ms/step - loss: 13.5233 - val_loss: 280.8378
Epoch 1175/10000
4/4 [==============================] - 0s 12ms/step - loss: 13.2152 - val_loss: 287.2607
Epoch 1176/10000
4/4 [==============================] - 0s 12ms/step - loss: 12.0029 - val_loss: 278.6586
Epoch 1177/10000
4/4 [==============================] - 0s 12ms/step - loss: 15.5646 - val_loss: 278.6869
Epoch 1178/10000
4/4 [==============================] - 0s 11ms/

In [160]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [161]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,88.275299,86.634109,86.87001,87.732269,87.611015,84.184616,71.09713,70.520256,71.524864,75.949699,85.440559,87.895615,86.74054,87.144226,87.453995,87.415749,87.297394,86.845863,74.788773,71.126862,71.911224,75.67186,82.205894,86.478958,86.925888,86.04628,86.445953,86.561722,86.566948,79.156723
Target,88.78,86.195,84.121,88.187,79.514,74.061,64.818,44.254,52.169,53.399,45.364,67.103,61.062,73.445,69.471,83.004,77.332,58.591,70.66,50.796,47.834,45.327,54.032,60.657,67.25,70.773,67.01,81.128,81.737,76.841
Error,0.5047,0.43911,2.749008,0.454727,8.097015,10.123619,6.279129,26.266254,19.355865,22.550701,40.076561,20.792618,25.678539,13.699226,17.982994,4.411751,9.965393,28.254864,4.128769,20.33086,24.077225,30.34486,28.173893,25.821957,19.675888,15.273277,19.435951,5.433723,4.829948,2.31572


In [162]:
display(mae)
display(mape)

15.250806

0.26758698

In [163]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [164]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[983.7354]] - Target[827.965]| =  Error: [[155.77039]]; MAPE:[[0.18813643]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[985.0813]] - Target[752.2110000000002]| =  Error: [[232.8703]]; MAPE:[[0.3095811]]
1/1 [==============================] - 0s 17ms/step
Ano-5: |Prediction[[511.70355]] - Target[444.73900000000003]| =  Error: [[66.96454]]; MAPE:[[0.15057042]]


[array([[155.77039]], dtype=float32),
 array([[232.8703]], dtype=float32),
 array([[66.96454]], dtype=float32)]

151.86841

0.21609598